In [1]:
import gradio as gr
import pyodbc
import pandas as pd 
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from fuzzywuzzy import fuzz, process
from sklearn.neighbors import NearestNeighbors

server = 'DESKTOP-77HM0LQ'
database = 'DATA_For_ML'
username = 'sa'
password = 'taolakhaiden0285'

connection_String = f'DRIVER={{SQL Server}};SERVER={server};DATABASE={database};UID={username};PWD={password}'

try: 
    # Kết nối với databse và in kết quả 5 sản phẩm đầu tiên của bảng Product
    conn = pyodbc.connect(connection_String)
    query = 'SELECT * FROM Tiki_Product'
    
    df_sanpham = pd.read_sql(query, conn)
    print(df_sanpham.head(5))

except pyodbc.Error as e: 
    print(f'Error: {e}')
finally:
    if conn:
        conn.close()

# Tạo đặc trưng cho dữ liệu : 
features = ['Product_name', 'Price']
def combineFeatures(row):
    return str(row['Price']) + " " + str(row['Product_name'])

df_sanpham['combinedFeatures'] = df_sanpham.apply(combineFeatures, axis=1)
print(df_sanpham['combinedFeatures'].head())

tf = TfidfVectorizer()
tfMatrix = tf.fit_transform(df_sanpham['combinedFeatures'])

similar = cosine_similarity(tfMatrix)
number = 5 

def get_similar_products(Product_name):
    ketqua = []
    # Tìm kiếm sản phẩm gần giống
    Product_name_list = df_sanpham['Product_name'].tolist()
    best_match = process.extractOne(Product_name, Product_name_list)
    if best_match:
        Product_name = best_match[0]
        product_id = df_sanpham[df_sanpham['Product_name'] == Product_name]['Product_id'].values[0]
        indexproductid = df_sanpham[df_sanpham['Product_id'] == product_id].index[0]
        
        similarProduct = list(enumerate(similar[indexproductid]))

        sortedSimilarProduct = sorted(similarProduct, key=lambda x: x[1], reverse=True)

        def lay_ten(index): 
            return (df_sanpham[df_sanpham.index == index]['Product_name'].values[0])

        for i in range(1, number + 1): 
            ketqua.append(lay_ten(sortedSimilarProduct[i][0]))

        result = "Sản phẩm gợi ý:\n"
        for product in ketqua:
            result += product + "\n"
        return result
    else:
        return "Không tìm thấy sản phẩm"

def get_similar_products_by_category(Product_name):
    ketqua = []
    # Tìm kiếm sản phẩm gần giống
    Product_name_list = df_sanpham['Product_name'].tolist()
    best_match = process.extractOne(Product_name, Product_name_list)
    if best_match:
        Product_name = best_match[0]
        product_id = df_sanpham[df_sanpham['Product_name'] == Product_name]['Product_id'].values[0]
        category = df_sanpham[df_sanpham['Product_id'] == product_id]['Category'].values[0]
        
        # Lấy danh sách sản phẩm trong cùng một danh mục
        similar_products = df_sanpham[df_sanpham['Category'] == category]
        
        # Loại bỏ sản phẩm đã chọn
        similar_products = similar_products[similar_products['Product_name'] != Product_name]
        
        # Lấy 5 sản phẩm đầu tiên
        similar_products = similar_products.head(5)
        
        # Lấy danh sách tên sản phẩm
        ketqua = similar_products['Product_name'].tolist()
        
        result = "Sản phẩm gợi ý:\n"
        for product in ketqua:
            result += product + "\n"
        return result
    else:
        return "Không tìm thấy sản phẩm"

def collaborative_filtering(customer_id):
    # Đọc dữ liệu từ cơ sở dữ liệu
    query = 'SELECT * FROM Ratings'
    ratings = pd.read_sql(query, conn)

    # Tạo ma trận đánh giá trung bình
    user_item_matrix = ratings.pivot(index='Customer_id', columns='Product_id', values='Rating_avg')

    # Tạo mô hình nearest neighbors
    model = NearestNeighbors(n_neighbors=5, algorithm='brute', metric='cosine')
    model.fit(user_item_matrix.fillna(0))

    # Tìm kiếm sản phẩm gợi ý cho khách hàng
    distances, indices = model.kneighbors(user_item_matrix.iloc[customer_id-1].values.reshape(1, -1), n_neighbors=5)

    # Lấy danh sách sản phẩm gợi ```python
    # Lấy danh sách sản phẩm gợi ý
    recommended_products = []
    for index in indices[0]:
        product_id = user_item_matrix.columns[index]
        product_name = df_sanpham[df_sanpham['Product_id'] == product_id]['Product_name'].values[0]
        recommended_products.append(product_name)

    # Trả về danh sách sản phẩm gợi ý
    result = "Sản phẩm gợi ý:\n"
    for product in recommended_products:
        result += product + "\n"
    return result

demo = gr.Interface(
    fn=get_similar_products_by_category,
    inputs=gr.Textbox(label="Nhập tên sản phẩm"),
    outputs=gr.Textbox(label="Kết quả"),
    title="Gợi ý sản phẩm",
    description="Nhập tên sản phẩm để nhận gợi ý"
)

demo.launch()

c:\Users\LENOVO\AppData\Local\Programs\Python\Python310\lib\site-packages\fuzzywuzzy\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')
C:\Users\LENOVO\AppData\Local\Temp\ipykernel_3700\3408936081.py:21: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_sanpham = pd.read_sql(query, conn)


   Product_id                                       Product_name     Price  \
0       54645  Tai nghe chụp tai Logitech H150 - 2 jack 3.5mm...  289000.0   
1       54665    Chuột không dây Logitech M185 - Hãng chính hãng  219000.0   
2       56033    Chuột Không Dây Logitech M187 - Hàng chính hãng  265000.0   
3      104180  Loa Vi Tính SoundMax A-130/2.0 6W  - Hàng Chín...  241000.0   
4      122012  Combo chuột phím không dây Logitech MK240 - Hà...  480000.0   

   Original_price  Discount  Discount_rate  Rating_avg Inventory_status  \
0        328900.0   39900.0           12.0         4.5        available   
1        289000.0   70000.0           24.0         4.7        available   
2        265000.0       0.0            0.0         5.0        available   
3        266000.0   25000.0            9.0         5.0        available   
4        516000.0   36000.0            7.0         4.5        available   

  Brand_id                   Category                   Sub_category  
0    1898